In [1]:
import proto_design
import nest_asyncio

lv1_area = proto_design.fetch_subarea('360')

nest_asyncio.apply()
lv1_area_codes = [i['kodeWilayah'] for i in lv1_area if i['namaWilayah'] != 'LUAR NEGERI']
lv2_area = proto_design.crawl_subareas(lv1_area_codes)
lv2_area_codes = [i['kodeWilayah'] for i in lv2_area]
lv3_area = proto_design.crawl_subareas(lv2_area_codes)

level1= []
for prov in lv1_area:
    item = {}
    item['kodeProv'] = prov['kodeWilayah']
    item['kodeProvDagri'] = prov['kodeWilayahDagri']
    item['namaProv'] = prov['namaWilayahDagri'].strip()
    level1.append(item)

level2= []
for kabkot in lv2_area:
    item2 = {}
    item2['kodeKabKot'] = kabkot['kodeWilayah']
    item2['kodeKabKotDagri'] = kabkot['kodeWilayahDagri']
    item2['namaKabKot'] = kabkot['namaWilayahDagri'].strip()
    item2['kodeProv'] = kabkot['kodeIndukWilayah']
    level2.append(item2)

level3= []
for kec in lv3_area:
    item3 = {}
    item3['kodeKec'] = kec['kodeWilayah']
    item3['kodeKecDagri'] = kec['kodeWilayahDagri']
    item3['namaKec'] = kec['namaWilayahDagri'].strip()
    item3['kodeKabKot'] = kec['kodeIndukWilayah']
    level3.append(item3)

In [4]:
merged_test = []
for h in level3:
    for i in level2:
        for j in level1:
            if h['kodeKabKot'] == i['kodeKabKot'] and i['kodeProv'] == j['kodeProv']:
                merged_dict = h|i|j
            else:
                None
    merged_test.append(merged_dict)

In [11]:
import pandas as pd
jatim = [j for j in merged_test if j['namaProv'] == 'JAWA TIMUR']
pd.DataFrame(jatim).to_csv('kode_jatim.csv', index=False)

In [1]:
import csv
import proto_design
import nest_asyncio
import logging
import sys

logstream_handler = logging.StreamHandler(stream=sys.stdout)
logging.basicConfig(handlers=[logstream_handler],
                    format='%(asctime)s.%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

nest_asyncio.apply()

with open('kode_wilayah.csv', 'r') as f:
    wilayah = f.read().splitlines()

sel_wilayah = [i['kodeKec'] for i in csv.DictReader(wilayah)]

cuba = proto_design.crawl_schlists(sel_wilayah)

2024-08-03 00:08:42.461 httpx INFO HTTP Request: GET https://api.data.belajar.id/data-portal-backend/v1/master-data/satuan-pendidikan/statistics/060615 "HTTP/1.1 200 OK"
2024-08-03 00:08:42.527 httpx INFO HTTP Request: GET https://api.data.belajar.id/data-portal-backend/v1/master-data/satuan-pendidikan/statistics/060120 "HTTP/1.1 200 OK"
2024-08-03 00:08:42.529 httpx INFO HTTP Request: GET https://api.data.belajar.id/data-portal-backend/v1/master-data/satuan-pendidikan/statistics/060118 "HTTP/1.1 200 OK"
2024-08-03 00:08:42.531 httpx INFO HTTP Request: GET https://api.data.belajar.id/data-portal-backend/v1/master-data/satuan-pendidikan/statistics/061705 "HTTP/1.1 200 OK"
2024-08-03 00:08:42.532 httpx INFO HTTP Request: GET https://api.data.belajar.id/data-portal-backend/v1/master-data/satuan-pendidikan/statistics/060104 "HTTP/1.1 200 OK"
2024-08-03 00:08:42.534 httpx INFO HTTP Request: GET https://api.data.belajar.id/data-portal-backend/v1/master-data/satuan-pendidikan/statistics/06011

KeyboardInterrupt: 

In [2]:
cuba

[{'NPSN': '69866676',
  'Nama Satuan Pendidikan': 'KB SITI KHODIJAH',
  'Jenjang': 'KB/Sederajat',
  'Jenis': 'Anak Usia Dini',
  'Alamat': 'LEBAK',
  'Kelurahan': 'AROSBAYA',
  'Status': 'SWASTA',
  'kodeKec': '052913'},
 {'NPSN': '69866677',
  'Nama Satuan Pendidikan': 'KB ROBIATUL ADAWIYAH',
  'Jenjang': 'KB/Sederajat',
  'Jenis': 'Anak Usia Dini',
  'Alamat': 'JL. LIDAH BUMI NO. 90 DESA BERBELUK',
  'Kelurahan': 'BERBELUK',
  'Status': 'SWASTA',
  'kodeKec': '052913'},
 {'NPSN': '69866672',
  'Nama Satuan Pendidikan': 'KB PERMAHISA',
  'Jenjang': 'KB/Sederajat',
  'Jenis': 'Anak Usia Dini',
  'Alamat': 'DSN. MORSABE DS. GLAGGA',
  'Kelurahan': 'GLAGGA',
  'Status': 'SWASTA',
  'kodeKec': '052913'},
 {'NPSN': '69909144',
  'Nama Satuan Pendidikan': 'KB TARBIYATUL MUHTAJIN AN-NASHIR',
  'Jenjang': 'KB/Sederajat',
  'Jenis': 'Anak Usia Dini',
  'Alamat': 'DSN JANGLOR 1 DS. LAJING',
  'Kelurahan': 'LAJING',
  'Status': 'SWASTA',
  'kodeKec': '052913'},
 {'NPSN': '69969394',
  'Nama Sat

In [ ]:
import httpx
import asyncio

import logging

import sys


from proto_design import TARGET_BASE_URL

from httpx import ReadTimeout, RemoteProtocolError

logstream_handler = logging.StreamHandler(stream=sys.stdout)
logging.basicConfig(handlers=[logstream_handler],
                    format='%(asctime)s.%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

def paginator(items:list, items_per_page:int):
    pages = [items[item:item+items_per_page] for item in range(0, len(items), items_per_page)]
    return pages

async def _crawl(_area:str) -> list:
    async with httpx.AsyncClient(timeout=None) as client:
        # try:
            data_url = f'{TARGET_BASE_URL}satuan-pendidikan/download?kodeKecamatan={_area}&sortBy=bentuk_pendidikan&sortDir=asc&format=csv'
            fetch_raw = await client.get(data_url)
            resp = fetch_raw.content.decode('UTF-8')
            parse_data =  csv.DictReader(resp.splitlines())
            stacked = [i for i in parse_data]
            
            return stacked
        # except ReadTimeout:
        #     print(data_url)
        #     return exit(0)
        
        
async def job_aggregator(job_list):
    job_agg = await asyncio.gather(*job_list)
    return job_agg

sch_list = []
for subset in paginator(sel_wilayah, 50):
    joblist = [_crawl(area) for area in subset]
    sequences = asyncio.run(job_aggregator(joblist))
    sch_list.append(sequences)

In [4]:
unnest_data = [datum for array_datum in sch_list for datum in array_datum]

In [6]:
unnest_data2 = [datum for array_datum in unnest_data for datum in array_datum]

In [8]:
import pandas as pd
df = pd.DataFrame(unnest_data2)

In [1]:
import proto_design
proto_design.fetch_schlist('210905')

[{'NPSN': '69941953',
  'Nama Satuan Pendidikan': 'KB MEGA BUANA',
  'Jenjang': 'KB/Sederajat',
  'Jenis': 'Anak Usia Dini',
  'Alamat': 'DESA HOTE JAYA',
  'Kelurahan': 'HOTE JAYA',
  'Status': 'SWASTA',
  'kodeKec': '210905',
  'serverTimestamp': '2024-08-01T22:06:59.900494Z'},
 {'NPSN': '69922972',
  'Nama Satuan Pendidikan': 'PAUD KASIH BUNDA',
  'Jenjang': 'KB/Sederajat',
  'Jenis': 'Anak Usia Dini',
  'Alamat': 'DESA LENA',
  'Kelurahan': 'LENA',
  'Status': 'SWASTA',
  'kodeKec': '210905',
  'serverTimestamp': '2024-08-01T22:06:59.900494Z'},
 {'NPSN': '69961586',
  'Nama Satuan Pendidikan': 'KB BATU INDAH',
  'Jenjang': 'KB/Sederajat',
  'Jenis': 'Anak Usia Dini',
  'Alamat': 'DESA POHON BATU',
  'Kelurahan': 'POHON BATU',
  'Status': 'SWASTA',
  'kodeKec': '210905',
  'serverTimestamp': '2024-08-01T22:06:59.900494Z'},
 {'NPSN': '69905059',
  'Nama Satuan Pendidikan': 'KB TUNAS BARU',
  'Jenjang': 'KB/Sederajat',
  'Jenis': 'Anak Usia Dini',
  'Alamat': 'DESA SIMI',
  'Kelurahan